In [1]:
# https://keras.io/
#!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [26]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [24]:
#num_filter = 12
num_filter = 32
dropout_rate = 0.2
l = 12

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
#Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [25]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_235 (Conv2D)             (None, 32, 32, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_235 (BatchN (None, 32, 32, 32)   128         conv2d_235[0][0]                 
__________________________________________________________________________________________________
activation_235 (Activation)     (None, 32, 32, 32)   0           batch_normalization_235[0][0]    
__________________________________________________________________________________________________
conv2d_236

In [27]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [28]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 238s 5ms/step - loss: 1.9049 - acc: 0.2759 - val_loss: 2.2685 - val_acc: 0.2728
Epoch 2/50
50000/50000 [==============================] - 218s 4ms/step - loss: 1.6320 - acc: 0.3805 - val_loss: 1.7733 - val_acc: 0.3691
Epoch 3/50
50000/50000 [==============================] - 218s 4ms/step - loss: 1.2114 - acc: 0.5596 - val_loss: 1.3249 - val_acc: 0.5345
Epoch 8/50
50000/50000 [==============================] - 218s 4ms/step - loss: 1.1593 - acc: 0.5769 - val_loss: 1.1583 - val_acc: 0.5908
Epoch 9/50
50000/50000 [==============================] - 218s 4ms/step - loss: 1.1158 - acc: 0.5970 - val_loss: 1.4052 - val_acc: 0.5198
Epoch 10/50
50000/50000 [==============================] - 218s 4ms/step - loss: 1.0788 - acc: 0.6096 - val_loss: 1.2934 - val_acc: 0.5811
Epoch 11/50
50000/50000 [==============================] - 218s 4ms/step - loss: 1.0434 - acc: 0.6234 - val_loss: 1.1367 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



50000/50000 [==============================] - 218s 4ms/step - loss: 0.6149 - acc: 0.7817 - val_loss: 0.7820 - val_acc: 0.7523
Epoch 42/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.6108 - acc: 0.7836 - val_loss: 1.2390 - val_acc: 0.6586
Epoch 43/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.6068 - acc: 0.7856 - val_loss: 0.8225 - val_acc: 0.7458
Epoch 44/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5946 - acc: 0.7901 - val_loss: 0.8005 - val_acc: 0.7501
Epoch 45/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5876 - acc: 0.7925 - val_loss: 0.7702 - val_acc: 0.7621
Epoch 46/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5816 - acc: 0.7952 - val_loss: 1.5263 - val_acc: 0.6050
Epoch 47/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5740 - acc: 0.7958 - val_loss: 0.7503 - val_acc: 0.7638
Epoch 48/50
50000/50000 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# We have ran the model for 50 epocs validation accuracy is around 75

In [29]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 15s 1ms/step
Test loss: 0.8630971043586732
Test accuracy: 0.7352


In [30]:
from keras.callbacks import ModelCheckpoint

In [32]:
#filepath = file_prefix + r".best.hdf5"
filepath = "densenet-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [33]:
# Save the trained weights in to .h5 format
model.save_weights("densenet-{epoch:02d}-{val_acc:.2f}.hdf5")
print("Saved model to disk")

Saved model to disk


In [0]:
#from google.colab import files

#files.download('DNST_model.h5')

In [34]:
callbacks_list = [checkpoint]

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5481 - acc: 0.8068 - val_loss: 0.8859 - val_acc: 0.7329

Epoch 00001: val_acc improved from -inf to 0.73290, saving model to densenet-01-0.73.hdf5
Epoch 2/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5417 - acc: 0.8087 - val_loss: 0.7878 - val_acc: 0.7589

Epoch 00002: val_acc improved from 0.73290 to 0.75890, saving model to densenet-02-0.76.hdf5
Epoch 3/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5372 - acc: 0.8082 - val_loss: 0.7391 - val_acc: 0.7770

Epoch 00003: val_acc improved from 0.75890 to 0.77700, saving model to densenet-03-0.78.hdf5
Epoch 4/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5322 - acc: 0.8104 - val_loss: 0.7139 - val_acc: 0.7774

Epoch 00004: val_acc improved from 0.77700 to 0.77740, saving model to densenet-04-0.78.hdf5
Epoch 5/50
21248/50000 [=

# ran the model for another 4 epochs,
# Highest validation accuracy is 0.77
# Total epochs ran = 54

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3575 - acc: 0.8739 - val_loss: 0.7500 - val_acc: 0.7998

Epoch 00001: val_acc did not improve from 0.81130
Epoch 2/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3542 - acc: 0.8761 - val_loss: 0.7343 - val_acc: 0.7993

Epoch 00002: val_acc did not improve from 0.81130
Epoch 3/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3531 - acc: 0.8749 - val_loss: 0.6764 - val_acc: 0.8116

Epoch 00003: val_acc improved from 0.81130 to 0.81160, saving model to densenet-03-0.81.hdf5
Epoch 4/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3472 - acc: 0.8772 - val_loss: 0.7236 - val_acc: 0.8018

Epoch 00004: val_acc did not improve from 0.81160
Epoch 5/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3487 - acc: 0.8770 - val_loss: 0.7087 - val_acc: 0.8135

Epoch 00005:

# ran the model for another 11 epochs,
# Highest validation accuracy is 0.82
# Total epochs ran = 65